## Step 1: Dataset 준비 및 사전 작업

In [2]:
# 필요한 라이브러리 설치 및 불러오기
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

# DistilBERT pre-trained tokenizer 불러오기
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

# AG_News 데이터셋 로드
ds = load_dataset("fancyzhx/ag_news")

# Truncation을 제외한 collate_fn 함수 정의
def collate_fn(batch):
    texts, labels = [], []
    for row in batch:
        labels.append(row['label'])
        texts.append(row['text'])

    encoding = tokenizer(texts, padding=True, return_tensors="pt")
    input_ids = encoding["input_ids"]
    attention_mask = encoding["attention_mask"]
    labels = torch.LongTensor(labels)

    return input_ids, attention_mask, labels

# DataLoader 설정
train_loader = DataLoader(ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn)

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://h

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

## Step 2: 모델 정의 및 수정

In [5]:
from torch import nn

# DistilBERT를 사용하는 텍스트 분류 모델 정의
class TextClassifier(nn.Module):
    def __init__(self):
        super(TextClassifier, self).__init__()
        self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
        self.classifier = nn.Linear(768, 4)  # 4개의 클래스

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs['last_hidden_state']
        x = self.classifier(x[:, 0])  # [CLS] 토큰만 사용하여 분류
        return x

model = TextClassifier().to('cuda')

# CrossEntropyLoss 사용 (다중 클래스 분류)
loss_fn = nn.CrossEntropyLoss()


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


## 3. 함수 수정 및 평가

In [6]:
from torch.optim import Adam

# Optimizer 설정
optimizer = Adam(model.parameters(), lr=0.0001)  # 학습률 줄임

# Accuracy 함수 수정 (다중 클래스 분류에 맞게)
def accuracy(model, dataloader):
    cnt = 0
    acc = 0
    model.eval()

    with torch.no_grad():
        for data in dataloader:
            input_ids, attention_mask, labels = data
            input_ids, attention_mask, labels = input_ids.to('cuda'), attention_mask.to('cuda'), labels.to('cuda')

            preds = model(input_ids, attention_mask)
            preds = torch.argmax(preds, dim=-1)  # 가장 높은 확률의 클래스를 예측

            cnt += labels.size(0)
            acc += (preds == labels).sum().item()

    return acc / cnt

# 학습 루프 (매 epoch마다 train loss 출력)
n_epochs = 10
for epoch in range(n_epochs):
    total_loss = 0
    model.train()

    for data in train_loader:
        input_ids, attention_mask, labels = data
        input_ids, attention_mask, labels = input_ids.to('cuda'), attention_mask.to('cuda'), labels.to('cuda')

        optimizer.zero_grad()
        preds = model(input_ids, attention_mask)
        loss = loss_fn(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1:2d} | Train Loss: {total_loss:.4f}")

# 최종 학습 후 정확도 측정
train_acc = accuracy(model, train_loader)
test_acc = accuracy(model, test_loader)
print(f"=========> Train Accuracy: {train_acc:.3f} | Test Accuracy: {test_acc:.3f}")


Epoch  1 | Train Loss: 407.3806
Epoch  2 | Train Loss: 246.2814
Epoch  3 | Train Loss: 171.6969
Epoch  4 | Train Loss: 122.9436
Epoch  5 | Train Loss: 89.0730
Epoch  6 | Train Loss: 72.3245
Epoch  7 | Train Loss: 54.9770
Epoch  8 | Train Loss: 50.3001
Epoch  9 | Train Loss: 45.6764
Epoch 10 | Train Loss: 36.9729
=========> Train Accuracy: 0.997 | Test Accuracy: 0.934


## 샘플 출력

In [7]:
import random

# 예측 결과 샘플 출력 함수
def print_sample_predictions(model, dataloader, num_samples=10):
    model.eval()
    with torch.no_grad():
        for data in dataloader:
            input_ids, attention_mask, labels = data
            input_ids, attention_mask, labels = input_ids.to('cuda'), attention_mask.to('cuda'), labels.to('cuda')

            preds = model(input_ids, attention_mask)
            preds = torch.argmax(preds, dim=-1)

            # 랜덤으로 샘플 선택하여 예측 결과 출력
            for i in range(num_samples):
                idx = random.randint(0, input_ids.shape[0] - 1)
                print(f"Sample {i + 1}:")
                print(f"Text: {tokenizer.decode(input_ids[idx], skip_special_tokens=True)}")
                print(f"Prediction: {preds[idx].item()} | Actual: {labels[idx].item()}")
                print('-' * 50)
            break  # 한 배치에서만 샘플 추출

# 샘플 예측 출력
print_sample_predictions(model, test_loader)


Sample 1:
Text: johnson back to his best as d - backs end streak new york ( reuters ) - randy johnson struck out 14 batters in 8 1 / 3 innings to help the arizona diamondbacks end a nine - game losing streak with a 2 - 0 win over the host new york mets in the national league sunday.
Prediction: 1 | Actual: 1
--------------------------------------------------
Sample 2:
Text: storage, servers bruise hp earnings update earnings per share rise compared with a year ago, but company misses analysts'expectations by a long shot.
Prediction: 3 | Actual: 3
--------------------------------------------------
Sample 3:
Text: card fraud unit nets 36, 000 cards in its first two years, the uk's dedicated card fraud unit, has recovered 36, 000 stolen cards and 171 arrests - and estimates it saved 65m.
Prediction: 3 | Actual: 3
--------------------------------------------------
Sample 4:
Text: spam suspension hits sohu. com shares ( ft. com ) ft. com - shares in sohu. com, a leading us - listed chinese 

모델을 10 epoch 동안 학습한 결과, **Train Accuracy: 99.7%**, **Test Accuracy: 93.4%**로 높은 성능을 기록했습니다. 이를 통해 모델이 뉴스 기사 분류 작업에서 매우 좋은 성능을 보이는 것을 확인할 수 있습니다.